In [47]:
import json
import string
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory


import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to C:\Users\Judan Syamsul
[nltk_data]     Hadad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Load json file
f = open('intent/intent.json', 'r')
intent_json = json.load(f)

In [6]:
# Create list from json
input = []
intent = []

for i in range(len(intent_json['intents'])):
    for user_input in intent_json['intents'][i]['input']:
        input.append(user_input)
        intent.append(intent_json['intents'][i]['intent'])

In [7]:
# Create dataframe from json
df = pd.DataFrame({
    'input': input,
    'intent' : intent
    # 'response' : response
})
df.head()

,input,intent
0,dimana cari skck?,skck
1,bagaimana cara buat skck?,skck
2,apa itu skck?,skck
3,apakah skck saya masih berlaku?,skck
4,berapa lama skck saya bisa digunakan?,skck


In [8]:
# Define stemmer and stopword remover
factory = StemmerFactory()
stemmer = factory.create_stemmer()

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

In [9]:
slang = pd.read_csv('lexicon/slang ke semi baku.csv')

slang_replace = {}
for i, row in enumerate(slang['slang']):
    slang_replace[row] = slang['formal'].iloc[i]

In [10]:
baku = pd.read_csv('lexicon/slang ke baku.csv')

baku_replace = {}
for i, row in enumerate(baku['slang']):
    baku_replace[row] = baku['baku'].iloc[i]

In [11]:
# Create text cleaning function
def clean_text(text):
    new_text = []

    text = text.lower()

    for kata in text.split():
        if kata not in (slang_replace|baku_replace):
            new_text.append(kata)
        elif kata in baku_replace:
            new_text+=baku_replace[kata].split()
        elif kata in slang_replace:
            new_text+=slang_replace[kata].split()
    new_text = ' '.join(
        stemmer.stem(
            baku_replace.get(
                word,
                word
            )
        ) for word in new_text if word not in stopwords
    )

    new_text = new_text.translate(
        str.maketrans(
            '',
            '',
            string.punctuation
        )
    )
    
    return new_text

In [12]:
kalimat = 'Aku laper banget gatau maunya makan jeruk apa lagi meniru-niru... daah ngelamar'
clean_text(kalimat)

'aku lapar banget tahu mau makan jeruk apa tiru deh lamar'

In [13]:
df['clean_input'] = df['input'].apply(clean_text)
df['clean_input'].head()

0                   cari skck
1    bagaimana cara buat skck
2                    apa skck
3                   skck laku
4       berapa lama skck guna
Name: clean_input, dtype: object

In [18]:
# Create corpus
words = set([
    word for word in df['clean_input'] for word in word_tokenize(word)
    ]
)

In [19]:
corpus_size = len(words)

In [20]:
# Calculate word length of each row
df['length'] = df['clean_input'].apply(word_tokenize).apply(len)

In [21]:
sequence_length = int(round(df['length'].max(),0))

In [22]:
corpus_size, sequence_length

(248, 8)

In [23]:
# Label encoding
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_train = le.fit_transform(df['intent'])
y_train = to_categorical(y_train)

In [24]:
le.classes_

array(['biaya', 'daftar', 'dokumen', 'error upload', 'ktp', 'link sosmed',
       'lowongan', 'lupa password', 'nama_bot', 'pas foto', 'pengantar',
       'penutup', 'qualification', 'responsibilities-DS', 'salary',
       'sertifikat', 'skck', 'timeline', 'training', 'transkrip-ijazah'],
      dtype=object)

In [25]:
len(le.classes_)

20

In [31]:
textvect = tf.keras.layers.TextVectorization(
    max_tokens=corpus_size,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=sequence_length
)
textvect.adapt(df['clean_input'])

In [32]:
embedding = tf.keras.layers.Embedding(
    input_dim=corpus_size,
    output_dim=16,
    input_length=sequence_length,
    embeddings_initializer='uniform'
)

In [33]:
tes = 'saya mau daftar rekrutmen'
textvect(clean_text(tes))

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([23, 12,  9,  0,  0,  0,  0,  0], dtype=int64)>

In [34]:
embedding(textvect(clean_text(tes)))

<tf.Tensor: shape=(8, 16), dtype=float32, numpy=
array([[-0.02542248,  0.00998279, -0.03977418,  0.0229227 ,  0.02499067,
        -0.02349997,  0.02289952, -0.03516664,  0.0185622 ,  0.01135553,
        -0.02150436, -0.02754229,  0.03413017, -0.03732336, -0.03473509,
        -0.03962176],
       [-0.03841554, -0.0339941 , -0.00592101, -0.00311812, -0.02383698,
         0.02149773, -0.04878286,  0.02057935, -0.01832103, -0.04783771,
        -0.0266579 ,  0.04651934, -0.0121294 ,  0.03836897, -0.00749832,
        -0.02303582],
       [ 0.02678985,  0.02948945, -0.01625627, -0.02809404, -0.01418747,
        -0.02577124,  0.0125531 ,  0.04874678,  0.03672271,  0.03259566,
        -0.04959005,  0.03108711, -0.01820128, -0.04516334,  0.02067326,
         0.01959218],
       [ 0.03421601, -0.0026141 , -0.00448287, -0.01709114, -0.0223838 ,
         0.04017873,  0.00644184, -0.03445064, -0.0274464 ,  0.01546094,
         0.02669176,  0.04261522, -0.02345103,  0.02969506,  0.00325789,
         

In [35]:
# Create model
input = tf.keras.layers.Input(shape=(1,), dtype='string')
hidden_layer = textvect(input)
hidden_layer = embedding(hidden_layer)
hidden_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16))(hidden_layer)
output = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(hidden_layer)
model = tf.keras.Model(inputs=input, outputs=output)

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)

In [36]:
hist = model.fit(df['clean_input'], y_train, epochs=100, verbose=0)
pd.DataFrame(hist.history).tail()

,loss,categorical_accuracy
95,0.026891,1.0
96,0.026414,1.0
97,0.025692,1.0
98,0.025105,1.0
99,0.024784,1.0


In [37]:
model.evaluate(df['clean_input'], y_train)

12/12 [==============================] - 1s 3ms/step - loss: 0.0240 - categorical_accuracy: 1.0000


[0.023990172892808914, 1.0]

In [38]:
y_train_df = pd.DataFrame(y_train, columns=le.classes_)
y_train_df['intent'] = y_train_df.idxmax(axis=1)

model_pred = model.predict(df['clean_input'])
model_pred = pd.DataFrame(model_pred, columns=le.classes_)
model_pred['intent'] = model_pred.idxmax(axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_train_df['intent'], model_pred['intent']))

                     precision    recall  f1-score   support

              biaya       1.00      1.00      1.00        27
             daftar       1.00      1.00      1.00        22
            dokumen       1.00      1.00      1.00        22
       error upload       1.00      1.00      1.00         9
                ktp       1.00      1.00      1.00        15
        link sosmed       1.00      1.00      1.00        10
           lowongan       1.00      1.00      1.00        12
      lupa password       1.00      1.00      1.00        21
           nama_bot       1.00      1.00      1.00         8
           pas foto       1.00      1.00      1.00        15
          pengantar       1.00      1.00      1.00        10
            penutup       1.00      1.00      1.00        21
      qualification       1.00      1.00      1.00        17
responsibilities-DS       1.00      1.00      1.00        24
             salary       1.00      1.00      1.00        35
         sertifikat    

In [39]:
def bot_response(text):
    """Take text as function input then predict using model. Return response based on highest probability using numpy argmax    
    """
    text = clean_text(text)
    pred = model.predict([text])
    res = le.classes_[pred.argmax()]
    if textvect(text).numpy().max() > 1:
        for label_pred in intent_json['intents']:
            if label_pred['intent'] == res:
                response = label_pred['response']
    else:
        response = ['Maaf, saya tidak mengerti']
    
    dict_temp = []
    for i in range(len(pred[0])):
        temp = {le.classes_[i]: pred[0][i]}
        dict_temp.append(temp)
    print(dict_temp)
    print(le.classes_[pred.argmax()])
    return print(np.random.choice(response))

In [46]:
# import pickle
# pickle.dump(le, open('encoder.pkl', 'wb'))

In [48]:
# pickle.dump({'config': textvect.get_config(),
#              'weights': textvect.get_weights()}
#             , open("textvect.pkl", "wb"))

In [45]:
# tf.keras.models.save_model(model, 'model')

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [42]:
bot_response('laper pengen makan jeruk')

[{'biaya': 0.00032070838}, {'daftar': 0.0040558334}, {'dokumen': 8.260318e-06}, {'error upload': 0.0024041194}, {'ktp': 1.0298207e-05}, {'link sosmed': 0.00079323153}, {'lowongan': 0.0026841625}, {'lupa password': 0.00012575349}, {'nama_bot': 0.007540458}, {'pas foto': 0.010254186}, {'pengantar': 0.00026598954}, {'penutup': 6.285351e-05}, {'qualification': 0.0005937552}, {'responsibilities-DS': 0.0008006493}, {'salary': 0.17728098}, {'sertifikat': 0.2075185}, {'skck': 0.5413596}, {'timeline': 1.2068264e-05}, {'training': 0.0015309599}, {'transkrip-ijazah': 0.042377617}]
skck
Maaf, saya tidak mengerti


In [43]:
bot_response('pas foto hilang gimana ya')

[{'biaya': 6.7926056e-05}, {'daftar': 0.0011429955}, {'dokumen': 6.343329e-07}, {'error upload': 0.025918745}, {'ktp': 0.00012222306}, {'link sosmed': 3.411949e-05}, {'lowongan': 0.00016333196}, {'lupa password': 3.379874e-05}, {'nama_bot': 0.0014009523}, {'pas foto': 0.9626557}, {'pengantar': 0.0002479675}, {'penutup': 4.560631e-05}, {'qualification': 8.377216e-06}, {'responsibilities-DS': 0.00023101586}, {'salary': 0.002820793}, {'sertifikat': 0.0003212315}, {'skck': 0.0043877778}, {'timeline': 0.0001165808}, {'training': 0.00025660978}, {'transkrip-ijazah': 2.3711224e-05}]
pas foto
Pas foto merupakan dokumen wajib pendaftaran ya, Kak. Berikut ketentuan pas foto yang harus dipenuhi:
- Background berwarna biru dengan pakaian formal (kemeja dan/atau jas)
- Foto berukuran 3x4 dan berwarna
- Disarankan untuk menggunakan foto terbaru
- Upload foto di https://rekrutmen.fiktif.id/dokumen dengan ukuran file tidak lebih dari 1 MB dengan format file jpg/png/jpeg


ktp, pas foto, password masih ketuker2

In [44]:
le.classes_

array(['biaya', 'daftar', 'dokumen', 'error upload', 'ktp', 'link sosmed',
       'lowongan', 'lupa password', 'nama_bot', 'pas foto', 'pengantar',
       'penutup', 'qualification', 'responsibilities-DS', 'salary',
       'sertifikat', 'skck', 'timeline', 'training', 'transkrip-ijazah'],
      dtype=object)